In [1]:
# 각각의 neuron은 weight를 갖고있다.
# 이 때 weight는 이전 layer의 node를 위한 weight와 bias를 위한 weight(1개)로 구성된다.
# 따라서 아래 예시에서도 input_node의 수 + 1 만큼의 weight를 생성한 것이다. (hidden layer에서)

# hidden layer의 weight의 수 = input node의 수 + 1(for bias)
# output layer의 weight의 수 = hidden node의 수 + 1(for bias)

In [2]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    # input layer와 hidden layer를 연결하는 가중치
    # n_inputs + 1을 해준 이유는 input neuron에 대한 weight + bias에 대한 weight
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    # hidden layer와 output layer를 연결하는 가중치 -> 따라서 hidden layer의 neuron의 수 + 1(for bias)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2) 
# input layer의 node 2개 - hidden layer의 node 1개 - output layer의 node 2개
for layer in network:
    print(layer)

# network의 첫번째 딕셔너리에는 input layer와 hidden layer를 연결하는 weight
# network의 두번째 딕셔너리에는 hidden layer와 output layer를 연결하는 weight

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [3]:
# linear 연산 수행
def activate(weights, inputs):
    activation = weights[-1]
    # 위에서 이미 bias는 더해주었기 때문에 아래에서는 range(len(weights)-1)만큼만 돈다
    for i in range(len(weights)-1):
        activation += inputs[i]*weights[i]  # linear combination 연산 수행
    return activation

# activation function
def sigmoid(activation):
    return 1/(1+np.exp(-activation)) # 시그모이드 구현

# 순전파
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            # 이전 layer의 neuron과 다음 layer와의 neuron을 연결하는 가중치의 선형결합
            activation = activate(neuron['weights'], inputs)  
            # 위에서의 연산 결과가 activation function(여기서는 sigmoid)을 통과
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요? 
            # 그 결과를 new_inputs에 추가
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
        # 위에서 구한 new_inputs를 inputs으로 넘겨주고, 이 inputs는 다음 layer 연산에서 사용된다.
        inputs = new_inputs
    return inputs

**Q. 여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

A. 이 과정에서는 가중치가 **업데이트 되지 않는다.**  
단순히 sigmoid(weight_vector*input_vector) 결과를 다시 다음 layer로 넘겨주는 연산만 수행한다.   
함수에서도 network의 weights가 업데이트 되는 코드는 찾아볼 수 없다.

In [4]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [5]:
def sigmoid_derivative(output):
    # 시그모이드 활성화함수를 통과한 output에 대한 미분값
    return output*(1.0-output) # 시그모이드 미분

def backward_propagate_error(network, expected):
    # 마지막의 layer부터 미분값을 계산해나간다. -> 그래서 reversed 사용
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        # 마지막의 layer가 아닌 경우
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                # 바로 앞의 layer 정보 받아온다
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        # 마지막의 layer인 경우
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                # 밑에서 gradient descent를 += 형식으로 업데이트 하는 것으로 보아
                # 오차 미분한 것을 -(실제-예측)가 아니라 (실제-예측)로 표기해야 맞다.
                errors.append(expected[j]-neuron['output']) # 역전파시 오차는 어떻게 설정했나요?
        for j in range(len(layer)):
            neuron = layer[j]
            # 오차를 weight에 대해 편미분한 결과를 key delta에 저장한다.
            neuron['delta'] = errors[j]*sigmoid_derivative(neuron['output']) # 시그모이드 함수를 사용한 역전파 

In [6]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [7]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1] # feature 데이터
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]] # 이전 layer에서의 결과
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate* neuron['delta']  # 퍼셉트론 학습 규칙
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        # 하나의 데이터 추출
        for row in train:
            # 하나의 데이터에 대해서 outputs 계산
            outputs = forward_propagate(network, row) # 순전파 
            # target을 one-hot vector로 표시하기 위한 코드
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            # 예측값의 오차 합
            # (실제-예측)^2의 합이 오차
            sum_error += sum( [(expected[i] - outputs[i])**2 for i in range(len(expected))] ) / 2 
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [8]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
           [1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],
           [1.38807019,1.850220317,0],
           [3.06407232,3.005305973,0],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1],
           [8.675418651,-0.242068655,1],
           [7.673756466,3.508563011,1]]

In [9]:
n_inputs = 2 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
n_outputs = 2 # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 

In [10]:
network = initialize_network(n_inputs, 11, n_outputs)
train_network(network, dataset, 1, 40, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.
for neuron in network:
    print(neuron)

>epoch=0, lrate=1.000, error=4.976
>epoch=1, lrate=1.000, error=4.971
>epoch=2, lrate=1.000, error=4.963
>epoch=3, lrate=1.000, error=4.948
>epoch=4, lrate=1.000, error=4.908
>epoch=5, lrate=1.000, error=4.544
>epoch=6, lrate=1.000, error=3.267
>epoch=7, lrate=1.000, error=3.409
>epoch=8, lrate=1.000, error=3.296
>epoch=9, lrate=1.000, error=3.172
>epoch=10, lrate=1.000, error=3.048
>epoch=11, lrate=1.000, error=2.914
>epoch=12, lrate=1.000, error=2.712
>epoch=13, lrate=1.000, error=2.073
>epoch=14, lrate=1.000, error=1.430
>epoch=15, lrate=1.000, error=1.067
>epoch=16, lrate=1.000, error=0.807
>epoch=17, lrate=1.000, error=0.610
>epoch=18, lrate=1.000, error=0.462
>epoch=19, lrate=1.000, error=0.352
>epoch=20, lrate=1.000, error=0.272
>epoch=21, lrate=1.000, error=0.215
>epoch=22, lrate=1.000, error=0.173
>epoch=23, lrate=1.000, error=0.143
>epoch=24, lrate=1.000, error=0.121
>epoch=25, lrate=1.000, error=0.104
>epoch=26, lrate=1.000, error=0.091
>epoch=27, lrate=1.000, error=0.080
>e

In [11]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return np.argmax(outputs) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?

In [12]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
